## DAB200 -- Lab 7

In this lab, you will gain some experience in dealing with missing data and further practice converting non-numeric features in a dataset to numeric.

**Target**: to predict `Comb Unadj FE - Conventional Fuel`

**Data set**: will be assigned by the instructor in class

### Part 0

Please provide the following information by editing this cell:
 - Name:  Harshilkumar Patel
 - Student Number: 0790476

### Part 1 - Create and evaluate an initial model

In this part you should: 
 
 - read in the data
 - isolate all numeric features from original data set
 - fill in any missing values with 0
 - create and evaluate an initial model 
 - use 150 decision trees in your random forest model

#### Code (10 marks)

**Code for reading data:**

In [54]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import category_encoders as ce
from statistics import mean
from pandas.api.types import is_string_dtype, is_object_dtype
from pandas.api.types import is_categorical_dtype
data_string = 'veh16_missing.csv'
df1 = pd.read_csv(data_string)

**Code for exploring data:**

In [55]:
# baseline model
df_num = df1[['Eng Displ', '# Cyl', 'Comb Unadj FE - Conventional Fuel', '# Gears',
       'Max Ethanol % - Gasoline', 'Intake Valves Per Cyl',
       'Exhaust Valves Per Cyl']]
df_num = df_num.fillna(0) # fill the null value with 0
print(df_num.isnull().sum())
df_num['Comb Unadj FE - Conventional Fuel'] = df_num['Comb Unadj FE - Conventional Fuel'].replace('Mod',0)

Eng Displ                            0
# Cyl                                0
Comb Unadj FE - Conventional Fuel    0
# Gears                              0
Max Ethanol % - Gasoline             0
Intake Valves Per Cyl                0
Exhaust Valves Per Cyl               0
dtype: int64


#### Create and evaluate an initial model after isolating numeric features:
Put all your code inside the following function definition.  

Your solution must return:

- **first item:** Mean OOB score of 10 runs
- **second item:** The last (i.e., the 10th) random forest regressor object (fitted)
- **third item:** Feature array
- **fourth item:** Target array

In [56]:
def estimate_mean_oob_score_baseline():
    ### BEGIN SOLUTION
    oob_scores = []
    X = df_num.drop('Comb Unadj FE - Conventional Fuel',axis=1) # feature array
    y = df_num['Comb Unadj FE - Conventional Fuel'] # Target feature
    for i in range(10):
        rf = RandomForestRegressor(n_estimators=150, n_jobs=-1, oob_score=True)
        rf.fit(X,y)
        oob_scores.append(rf.oob_score_)
    return mean(oob_scores),rf,X,y
    ### END SOLUTION

**Running the following cell should not throw any error if your code in the cell above is correct.  Do not edit the following cell.**

In [57]:
baseline_oob_score, rf, X, y = estimate_mean_oob_score_baseline()

num_trees = len(rf.estimators_)

print(baseline_oob_score, num_trees)

if data_string == 'veh14_missing.csv':
    output = 0.696
elif  data_string == 'veh15_missing.csv':
    output = 0.730
elif  data_string == 'veh16_missing.csv':
    output = 0.738
elif  data_string == 'veh17_missing.csv':
    output = 0.721
elif  data_string == 'veh18_missing.csv':
    output = 0.673
elif  data_string == 'veh19_missing.csv':
    output = 0.704
    

if (np.isclose(baseline_oob_score , output, rtol = 0.01) == True) and (num_trees==150):
    part1_marks = 10
    
assert np.isclose(baseline_oob_score , output, rtol = 0.01)

0.738531602540271 150


### Part 2 - Normalize missing values for string data

In this part you should: 
 - use Section 7.4 of the textbook as a guide
 - convert **all** representations of missing data to a **single** representation
 
#### Code (15 marks)

**Code for exploring missing values:**

In [58]:
columns = ['Trans Creeper Gear', 'Unique Label?','Air Aspiration Method Desc', 'Fuel Metering Sys Desc', 'Cyl Deact?']
for i in columns:
    print(i,df1[i].unique())

Trans Creeper Gear ['N' 'Y']
Unique Label? ['N' 'Y' nan]
Air Aspiration Method Desc ['Naturally Aspirated' 'none given' 'Turbocharged' 'Supercharged'
 'Turbocharged+Supercharged']
Fuel Metering Sys Desc ['Multipoint/sequential fuel injection' 'Spark Ignition Direct Injection'
 '@@@@@' 'Spark Ignition direct & ported injection'
 'Common Rail Direct Diesel Injection'
 'Direct Diesel Injection (non-common rail)']
Cyl Deact? ['N' '   ' nan 'Y']


**Code for normalizing missing values:**

Put all your code inside the following function definition. 

In [59]:
def normalize_missing_values():
    ### BEGIN SOLUTION
    for col in df1.columns:
        if is_string_dtype(df1[col]) or is_object_dtype(df1[col]):
            df1[col] = df1[col].str.lower() # lower case
            df1[col] = df1[col].fillna(np.nan)
            df1[col] = df1[col].replace('nan', np.nan)
            df1[col] = df1[col].replace('none given', np.nan)
            df1[col] = df1[col].replace('@@@@@', np.nan)
            df1[col] = df1[col].replace('   ', np.nan)
    return df1
    
    ### END SOLUTION

**Running the following cell should not throw any error if your code in the cell above is correct.  Do not edit the following cell.**

In [60]:
df = normalize_missing_values()

print(df.head())

vissing_mals = ['not provided', '##-##', '   ', 'none', 'not filled in', '^^', 'unknown', 'XXXXX', 
                'not specified', '*****', '@@@@@', 'none given', '%%%%%', 'missing', 'mod']

col_missing = []

for col in df.columns:
    col_missing.append(all(df[col].isin(vissing_mals)))
    
if sum(col_missing) == 0:
    part2_marks = 15
else:
    part2_marks = 0
    
assert sum(col_missing) == 0

   Eng Displ  # Cyl Comb Unadj FE - Conventional Fuel  # Gears  \
0        NaN      4                           32.8729        6   
1        NaN      4                           41.5766        6   
2        1.8      4                           42.3624        6   
3        3.5      6                           29.3963        6   
4        NaN      6                           25.4694        6   

   Max Ethanol % - Gasoline  Intake Valves Per Cyl  Exhaust Valves Per Cyl  \
0                      10.0                      2                       2   
1                      10.0                      2                       2   
2                      15.0                      2                       2   
3                      10.0                      2                       2   
4                      85.0                      2                       2   

  Trans Creeper Gear Unique Label? Air Aspiration Method Desc  \
0                  n             n        naturally aspirated   
1   

### Part 3 - Encoding Categorical features

In this part you should: 
 - use Section 7.5.1 as a guide
 - only use label encoding 
 - convert **all** non-numeric features to numeric 
 - handle any missing values
 
#### Code (25 marks)

**Provide your code to convert all non-numeric features to numeric using label encoding. Use only `pandas` for encoding (don't use `category encoders`). Make sure that missing values are encoded as zero. All your code must be inside the function below**

In [61]:
def label_encoding_non_numeric_cols():
    ### BEGIN SOLUTION
    df = normalize_missing_values()
    for col in df.columns:
        if is_string_dtype(df[col]):
            df[col] = df[col].astype('category').cat.as_ordered()
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1
    return df
    ### END SOLUTION

**Running the following cell should not throw any error if your code in the cell above is correct.  Do not edit the following cell.**

In [62]:
df = label_encoding_non_numeric_cols()

print(df.head())

from pandas.api.types import is_numeric_dtype

cols_are_numeric = []
for col in df.columns:
    if col != 'Comb Unadj FE - Conventional Fuel':
        cols_are_numeric.append(is_numeric_dtype(df[col]))
    
    
if (df.shape[1]) - 1 == sum(cols_are_numeric):
    part3_marks = 25
else:
    part3_marks = 0
    
assert (df.shape[1]) - 1 == sum(cols_are_numeric)

   Eng Displ  # Cyl  Comb Unadj FE - Conventional Fuel  # Gears  \
0        NaN      4                                580        6   
1        NaN      4                                839        6   
2        1.8      4                                853        6   
3        3.5      6                                439        6   
4        NaN      6                                243        6   

   Max Ethanol % - Gasoline  Intake Valves Per Cyl  Exhaust Valves Per Cyl  \
0                      10.0                      2                       2   
1                      10.0                      2                       2   
2                      15.0                      2                       2   
3                      10.0                      2                       2   
4                      85.0                      2                       2   

   Trans Creeper Gear  Unique Label?  Air Aspiration Method Desc  \
0                   1              1                        

### Part 4 - Numeric features

In this part you should: 
 - use Section 7.5.2 as a guide
 - handle any missing values
 
#### Code (30 marks)

Return only the dataframe

In [64]:
def fill_missing_vals_num():
    ### BEGIN SOLUTION
    df = label_encoding_non_numeric_cols()
    for colname in df.columns:
        df[colname].fillna(df[colname].median(), inplace=True)
    return df
    ### END SOLUTION

**Running the following cell should not throw any error if your code in the cell above is correct. Do not edit the following cell.**

In [65]:
df = fill_missing_vals_num()

cols_not_null = []
for col in df.columns:
    cols_not_null.append(sum(pd.isnull(df[col])))
    
if all(cols_not_null) == 0:
    part4_marks = 30
else:
    part4_marks = 0
    
assert all(cols_not_null) == 0

### Part 5 - Create and evaluate a final model

In this part you should:
 - create and evaluate a model using all the features after processing them in Parts 2, 3, and 4 above 
 - use 150 decision trees
 

The following function must return:
- **first item:** Mean OOB score of 10 runs
- **second item:** The last (i.e., the 10th) random forest regressor object (fitted)
- **third item:** Feature array
- **fourth item:** Target array

#### Code (10 marks)

In [71]:
def estimate_mean_oob_score_final():
    ### BEGIN SOLUTION
    df = fill_missing_vals_num()
    oob_scores = []
    X = df.drop('Comb Unadj FE - Conventional Fuel',axis=1) # feature array
    y = df['Comb Unadj FE - Conventional Fuel'] # Target feature
    for i in range(10):
        rf = RandomForestRegressor(n_estimators=150, n_jobs=-1, oob_score=True)
        rf.fit(X,y)
        oob_scores.append(rf.oob_score_)
    return mean(oob_scores),rf,X,y
    ### END SOLUTION

**Running the following cell should not throw any error if your code in the cell above is correct. Do not edit the following cell.**

In [72]:
final_oob_score, rf, X, y = estimate_mean_oob_score_final()

num_trees = len(rf.estimators_)

print(final_oob_score, num_trees)

if final_oob_score > baseline_oob_score:
    part5_marks = 10
else:
    part5_marks = 0
    
assert final_oob_score > baseline_oob_score

0.7995486297360186 150


In [73]:
parts_1_to_5_marks = part1_marks + part2_marks + part3_marks + part4_marks + part5_marks
parts_1_to_5_marks

90

In [77]:
df = fill_missing_vals_num()
df.rename(columns = {'Eng Displ' : 'Eng_Displ','# Cyl' : 'No_Cyl', '# Gears' : 'No_Gears', 'Max Ethanol % - Gasoline' : 'Max_Ethanol_Per_Gasoline',
              'Air Aspiration Method Desc' : 'Air_Aspiration_Method_Desc', 'Intake Valves Per Cyl' : 'Intake_Valves_Per_Cyl',
                'Exhaust Valves Per Cyl' : 'Exhaust_Valves_Per_Cyl', 'Trans Creeper Gear' : 'Trans_Creeper_Gear',
                'Unique Label?' : 'Unqiue_Labels', 'Fuel Metering Sys Desc' : 'Fuel_Metering_Sys_Desc', 'Cyl Deact?' : 'Cyl_Deact'})

,Eng_Displ,No_Cyl,Comb Unadj FE - Conventional Fuel,No_Gears,Max_Ethanol_Per_Gasoline,Intake_Valves_Per_Cyl,Exhaust_Valves_Per_Cyl,Trans_Creeper_Gear,Unqiue_Labels,Air_Aspiration_Method_Desc,Fuel_Metering_Sys_Desc,Cyl_Deact
0,3.0,4,580,6,10.0,2,2,1,1,1,3,1
1,3.0,4,839,6,10.0,2,2,1,1,0,5,1
2,1.8,4,853,6,15.0,2,2,1,1,0,3,0
3,3.5,6,439,6,10.0,2,2,1,1,0,5,0
4,3.0,6,243,6,85.0,2,2,1,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1208,3.0,8,151,8,15.0,2,2,1,1,0,4,0
1209,5.0,8,170,8,15.0,2,2,1,1,0,4,1
1210,3.3,6,288,6,10.0,2,2,1,1,1,5,1
1211,2.0,4,767,6,10.0,2,2,1,1,0,0,0


In [75]:
import pickle
pickle_out = open('rf.pkl','wb')
pickle.dump(rf,pickle_out)
pickle_out.close()

In [81]:
df.describe()

,Eng Displ,# Cyl,Comb Unadj FE - Conventional Fuel,# Gears,Max Ethanol % - Gasoline,Intake Valves Per Cyl,Exhaust Valves Per Cyl,Trans Creeper Gear,Unique Label?,Air Aspiration Method Desc,Fuel Metering Sys Desc,Cyl Deact?
count,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000
mean,3.037181,5.629019,451.331410,6.510305,15.453421,1.927453,1.921682,1.001649,1.090684,1.047815,2.901896,0.628195
std,1.117832,1.846649,264.404057,1.429048,17.216602,0.259499,0.268783,0.040589,0.457811,1.172860,2.121189,0.600684
min,0.900000,3.000000,1.000000,1.000000,10.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,4.000000,216.000000,6.000000,10.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,3.000000,6.000000,453.000000,6.000000,10.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,1.000000
75%,3.500000,6.000000,672.000000,8.000000,15.000000,2.000000,2.000000,1.000000,1.000000,2.000000,5.000000,1.000000
max,8.400000,12.000000,930.000000,9.000000,85.000000,2.000000,2.000000,2.000000,2.000000,4.000000,5.000000,2.000000
